In [1]:
import requests
import json
import xmltodict
import urllib.parse
import opencitingpy
import datetime
import sqlite3
import os
from matplotlib import pyplot as plt
import numpy as np
import re
import math
import ipywidgets as widgets
from IPython.display import display, clear_output
import re 
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from collections import Counter
from IPython.display import display
from Bio import Entrez
from matplotlib.ticker import MaxNLocator
import scipy
from scipy import stats
import mercury as mr

In [ ]:
# Anlegen von Datenbankspeicherort, falls er nicht existiert
path = "C:\MA_Pethke_3992454"
if not os.path.exists(path):
    mr.Markdown(f"## Es liegen noch keine Daten vor.")

# Verbindung zu DB aufbauen, DB wird automatisch erstellt, wenn keine da ist
con = sqlite3.connect("C:\MA_Pethke_3992454\MA_3992454_jp.db")
cur = con.cursor()

In [18]:
## Funktionen um Daten in DB einzupflegen

# Dokument Daten
def DB_Insert_Dokument(PMID, Titel, Jahr_Dok):
    try:
        cur.execute("INSERT INTO Dokument(PMID, Titel, Jahr_Dok) VALUES ('"+str(PMID)+"', '"+str(Titel)+"', '"+str(Jahr_Dok)+"')")
        con.commit()
        # Insert erfolgreich
    except:
        # Dokumenten-Daten schon in DB
        return
    
# Zitationsdaten für ein Dokument
def DB_Insert_Zitationen(values_str):
    try:
        cur.execute("INSERT INTO Zitationen(PMID, Jahr, Anzahl) VALUES " + str(values_str))
        con.commit()
        # Insert erfolgreich
    except:
        # Zitationsdaten schon in DB
        return
    
# MeSH Daten
def DB_Insert_MeSH_Daten(UI, Term, Baum, Jahr_MeSH):
    try:
        cur.execute("INSERT INTO MeSH(UI, Term, Baum, Jahr_MeSH) VALUES ('"+str(UI)+"', '"+str(Term)+"', '"+str(Baum)+"', '"+str(Jahr_MeSH)+"')")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH schon in DB
        return
    
# MeSH Analyse-Daten 
def DB_Insert_MeSH_Analyse(EK, NEK, NEKK, UI, prozent):
    try:
        # print("UPDATE MeSH SET EK = '"+str(EK)+"', NEK = '"+str(NEK)+"', NEKK = '"+str(NEKK)+"' WHERE UI ='" + str(UI) + "'")
        cur.execute("UPDATE MeSH SET EK = '"+str(EK)+"', NEK = '"+str(NEK)+"', NEKK = '"+str(NEKK)+"', EK_NEK = '" + str(prozent) + "' WHERE UI ='" + str(UI) + "'")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH Analysedaten schon in DB
        return    
    
# MeSH-Dokumenten Zuordnung
def DB_Insert_MeSH_PMID(UI, PMID):
    try:
        cur.execute("INSERT INTO MeSH_Dok(UI, PMID) VALUES ('"+str(UI)+"', '"+str(PMID)+"')")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH Analysedaten schon in DB
        return    
    
# MeSH-Dokument Analyse-Daten 
def DB_Insert_MeSH_Dokument_Analyse(EK_Dok, NEK_Dok, NEKK_Dok, UI, PMID, prozent):
    try:
        # print("UPDATE MeSH_Dok SET EK_Dok = '"+str(EK_Dok)+"', NEK_Dok = '"+str(NEK_Dok)+"', NEKK_Dok = '"+str(NEKK_Dok)+"' WHERE UI = '"+str(UI)+"' and PMID = '"+str(PMID)+"'")
        cur.execute("UPDATE MeSH_Dok SET EK_Dok = '"+str(EK_Dok)+"', NEK_Dok = '"+str(NEK_Dok)+"', NEKK_Dok = '"+str(NEKK_Dok)+"', EK_NEK_Dok = '" + str(prozent) + "' WHERE UI = '"+str(UI)+"' and PMID = '"+str(PMID)+"'")
        con.commit()
        # Insert erfolgreich
    except:
        # MeSH Analysedaten schon in DB
        return
    

def DB_Select_Alle_MeSH():
    cur.execute("SELECT Term FROM MeSH")
    MeSHs = [x[0] for x in cur.fetchall()]
    return MeSHs

def DB_Select_Alle_Dokumente_MeSH(meshUI):
    cur.execute("SELECT Titel FROM Dokument JOIN MeSH_Dok WHERE UI = '" + str(meshUI) +"'")
    Doks = [x[0] for x in cur.fetchall()]
    return Doks

def DB_Select_PMID_fromTitel(titel):
    cur.execute("SELECT PMID FROM Dokument WHERE Titel = '" + str(titel) +"'")
    return cur.fetchall()[0][0]

def DB_Select_Jahr_MeSH(UI):
    cur.execute("SELECT Jahr_MeSH FROM MeSH WHERE UI = '" + str(UI)+ "'")
    return cur.fetchall()[0][0]

def DB_Select_Term(UI):
    cur.execute("SELECT Term FROM MeSH WHERE UI = '" + str(UI)+ "'")
    return cur.fetchall()[0][0]

def DB_Select_Jahr_Dok(PMID):
    cur.execute("SELECT Jahr_Dok FROM Dokument WHERE PMID = '" + str(PMID)+ "'")
    return cur.fetchall()[0][0]

def DB_Select_Titel(PMID):
    cur.execute("SELECT Titel FROM Dokument WHERE PMID = '" + str(PMID)+ "'")
    return cur.fetchall()[0][0]


# Funktion, die prüft, ob MeSH Daten schon in DB
def pruefe_MeSH_in_DB(UI):
    cur.execute("SELECT count(1) FROM MeSH WHERE UI = '" + str(UI) + "'")
    Mesh_in_DB = cur.fetchall()[0][0]
    if Mesh_in_DB != 0:
        return 1 # Mesh ist schon in DB
    else:
        return 0 # Mesh ist noch nicht in DB

# Funktion, die prüft, ob Dokument schon in DB
def pruefe_Dok_in_DB(PMID):
    cur.execute("SELECT count(1) FROM Dokument WHERE PMID = '" + str(PMID) + "'")
    Dok_in_DB = cur.fetchall()[0][0]
    if Dok_in_DB != 0:
        return 1 # Dokument ist schon in DB
    else:
        return 0 # Dokument ist noch nicht in DB
    
def DB_Select_MeSH_UI(term):
    cur.execute("SELECT UI FROM MeSH WHERE Term = '" + str(term) + "'")
    MeSH_UI =  cur.fetchall()[0][0]
    return MeSH_UI

In [7]:
# Funktion zur Visualsierung der kumulierten Zitationszahlen
# Ausgabe von Eckdaten

# Funktion zur Visualisierung aller erhobenen Daten (Gemsatüberblick)
def visualisiereALLES():
    try:
        cur.execute("SELECT sum(EK), sum(NEK), sum(NEKK) FROM MeSH")
        y = cur.fetchall()[0]
        x = [0,1,2]
        maxZitation = max(y)
        if maxZitation > 0:
            fig, ax = plt.subplots(layout="constrained")
            ax.yaxis.set_major_locator(MaxNLocator(integer=True))
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))
            ax.set_ylabel("Anzahl Zitationen")
            ax.set_title("Gesamtüberblick")
            ax.set_xticks(ticks=x, labels=["Emergent", "Nicht-Emergent", "Nicht-Emergent Korrigiert"], rotation=45)
            ax.set_ylim(0, maxZitation+0.1*maxZitation)
            ax.bar(x, y)
            for i in range(len(x)):
                plt.text(i, y[i]+0.02*maxZitation, y[i], ha = 'center')
            ax.plot()
    except:
        print("Keine Daten vorhanden")


# Funktion zur Visualisierung der kumulierten Zitationsdaten zu einem MeSH
def visualisiereENENEKkumuliert_UI(UI):
    try:
        Term = DB_Select_Term(UI)
        cur.execute("SELECT EK, NEK, NEKK FROM MeSH WHERE UI = '"+ str(UI) + "'")
        y = cur.fetchall()[0]
        x = [0,1,2]
        maxZitation = max(y)
        if maxZitation > 0:
            fig, ax = plt.subplots(layout="constrained")
            ax.yaxis.set_major_locator(MaxNLocator(integer=True))
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))
            ax.set_ylabel("Anzahl Zitationen")
            ax.set_title("MeshUI: " + str(UI) + ", " + str(Term))
            ax.set_xticks(ticks=x, labels=["Emergent", "Nicht-Emergent", "Nicht-Emergent Korrigiert"], rotation=45)
            ax.set_ylim(0, maxZitation+0.1*maxZitation)
            ax.bar(x, y)
            for i in range(len(x)):
                plt.text(i, y[i]+0.02*maxZitation, y[i], ha = 'center')
            ax.plot()
    except:
        print("Keine Daten vorhanden")

# Funktion zur Visualsierung der Zitationen (E, NE, NEK) für ein bestimmtes Dokument zu einem bestimmten MeSH
def visualisiereENENEKkumuliert_UIPMID(UI, PMID):
    try:
        Term = DB_Select_Term(UI)
        Titel = DB_Select_Titel(PMID)
        cur.execute("SELECT EK_Dok, NEK_Dok, NEKK_Dok FROM MeSH_Dok WHERE UI = '"+ str(UI) + "' and PMID = '" + str(PMID) + "'")
        y = cur.fetchall()[0]
        x = [0,1,2]
        maxZitation = max(y)
        if maxZitation > 0:
            fig, ax = plt.subplots(layout="constrained")
            ax.yaxis.set_major_locator(MaxNLocator(integer=True))
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))
            ax.set_ylabel("Anzahl Zitationen")
            ax.set_title("MeshUI: " + str(UI) + ", " + str(Term) + "\n PMID: " +str(PMID) + ", " + str(Titel))
            ax.set_xticks(ticks=x, labels=["Emergent", "Nicht-Emergent", "Nicht-Emergent Korrigiert"], rotation=45)
            ax.set_ylim(0, maxZitation+0.1*maxZitation)
            ax.bar(x, y)
            for i in range(len(x)):
                plt.text(i, y[i]+0.02*maxZitation, y[i], ha = 'center')
            ax.plot()
    except:
        print("Keine Daten vorhanden")

# Funktion zur Visualisierung der Veröffentlichung, Mesh Einführung und Zitationshäufigkeiten
def visualisiereZitationenUndVeröffentlichung(PMID, UI):
    try:
        cur.execute("SELECT Jahr, Anzahl FROM Zitationen WHERE PMID = '" + str(PMID)+ "'")
        data = cur.fetchall()
        Jahr = [x[0] for x in data]
        Anzahl = [x[1] for x in data]
        maxZitation = max(Anzahl)
        print(int(0.2*maxZitation))
        cur.execute("SELECT Jahr_Dok FROM Dokument WHERE PMID = '" + str(PMID)+ "'")
        data = cur.fetchall()
        Jahr_Dok = data[0][0]
        cur.execute("SELECT Jahr_MeSH FROM MeSH WHERE UI = '" + str(UI)+ "'")
        data = cur.fetchall()
        Jahr_MeSH = data[0][0]
        if len(Jahr) != 0:
            fig, ax = plt.subplots(layout="constrained")
            ax.yaxis.set_major_locator(MaxNLocator(integer=True))
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))
            ax.set_ylabel("Anzahl Zitationen")
            ax.set_xlabel("Jahre")
            ax.set_title("PMID: " + str(PMID))
            ax.set_xlim(min(Jahr_Dok, Jahr_MeSH)-5, 2023)
            ax.set_ylim(0, maxZitation+0.1*maxZitation)
            ax.bar(Jahr, Anzahl)
            ax.annotate("Veröffentlichungsjahr",
                xy=(Jahr_Dok, int(0.2*maxZitation)), 
                xytext=(0, 0),
                textcoords="offset points", 
                arrowprops=dict(facecolor="blue"))
            ax.annotate("MeSH Aufnahme",
                xy=(Jahr_MeSH, int(0.4*maxZitation)), 
                xytext=(0, 0),
                textcoords="offset points", 
                arrowprops=dict(facecolor="red"))
            ax.plot()
    except:
        print("Keine Daten vorhanden")

In [ ]:
show_code = mr.Checkbox(label="Show code", value=True)
app = mr.App(title="Prozess: Visualisierung", 
             description="Visualisierung der Analyseergebnisse", show_code=show_code.value)
mr.Markdown(f"# Visualisierung von Zitationsdaten")

In [ ]:
alleMeSH_DB = DB_Select_Alle_MeSH()
mesh_auswahl_term = mr.Select(value=alleMeSH_DB[0], choices=alleMeSH_DB, label="Wählen Sie ein MeSH aus:")
mesh_auswahl_ui = DB_Select_MeSH_UI(mesh_auswahl_term.value)
doks_titel = DB_Select_Alle_Dokumente_MeSH(mesh_auswahl_ui)
dok_auswahl_titel = mr.Select(value=doks_titel[0], choices=doks_titel, label="Die in Verbindung stehenden Dokumente sind:")
dok_auswahl_pmid = DB_Select_PMID_fromTitel(dok_auswahl_titel.value)

In [ ]:
_ = visualisiereALLES()
_ = visualisiereENENEKkumuliert_UI(mesh_auswahl_ui)
_ = visualisiereZitationenUndVeröffentlichung(dok_auswahl_pmid, mesh_auswahl_ui)

